In [ ]:

import numpy as np
from ScanImageTiffReader import ScanImageTiffReader
import os
from ScanImageTiffReader import ScanImageTiffReader
import json
import ast

In [1]:
def find_tif_file(directory_path):
    # List all files in the directory
    files = os.listdir(directory_path)
    
    # Filter for .tif files
    tif_files = [f for f in files if f.endswith('.tif')]
    
    # Check if there is only one .tif file
    if len(tif_files) == 1:
        # Return the full path to the .tif file
        return os.path.join(directory_path, tif_files[0])
    elif len(tif_files) == 0:
        return "No .tif files found in the directory."
    else:
        return "Multiple .tif files found in the directory."
def parse_text_to_dict(text):
    # Split the text into lines and initialize an empty dictionary
    lines = text.split('\n')
    config_dict = {}
    json_str = ''
    json_started = False

    # Process each line
    for line in lines:
        # Handle JSON-like structures
        if line.strip().startswith('{'):
            json_started = True

        if json_started:
            json_str += line + '\n'
            if line.strip().endswith('}'):
                try:
                    json_dict = json.loads(json_str)
                    config_dict.update(json_dict)
                    json_started = False
                    json_str = ''
                except json.JSONDecodeError:
                    pass  # Continue accumulating lines for JSON
            continue

        # Process as key-value pairs
        if '=' in line:
            key, value = line.split('=', 1)
            key = key.strip()
            value = value.strip().strip("'")

            # Convert boolean and numeric values from string
            if value.lower() == 'true':
                value = True
            elif value.lower() == 'false':
                value = False
            elif value.replace('.', '', 1).isdigit():
                value = float(value) if '.' in value else int(value)

            config_dict[key] = value

    return config_dict

In [6]:
frames_metadata_export_folder = r'Z:\TM_Lab\Christy\HD2P'
frames_metadata_export_path = os.path.join(frames_metadata_export_folder, 'ca_img_frames_metadata.json')
img_path = os.path.join(frames_metadata_export_folder,'test_00006.tif')
try:
    with open(frames_metadata_export_path, 'r') as json_file:
        frames_metadata = json.load(json_file)
except:
    
    reader=ScanImageTiffReader(img_path)
    frames_metadata = []

    with ScanImageTiffReader(img_path) as reader:
        num_frames = reader.shape()[0]  # Assuming the first dimension represents frames
        
        for i in range(num_frames):
            description = reader.description(i)
            frame_metadata = {}  # Dictionary to hold this frame's metadata
            
            for line in description.split('\n'):
                if '=' in line:  # Ensure there's a key-value pair to parse
                    key, value = line.split('=', 1)  # Split by the first '='
                    key = key.strip()
                    
                    try:
                        # Safely evaluate the value string to the appropriate Python data type
                        # This works for numbers, lists, dictionaries, etc.
                        value = ast.literal_eval(value.strip())
                    except (ValueError, SyntaxError):
                        # If evaluation fails, keep the value as a string
                        value = value.strip()
                    
                    frame_metadata[key] = value
            
            frames_metadata.append(frame_metadata)
        
    with open(frames_metadata_export_path, 'w') as json_file:
        json.dump(frames_metadata, json_file)

In [8]:
auxTrigger = []
frame_time = []
trigger_frame_num = []
# Iterate through each frame's metadata
trig_chan = 'auxTrigger0'
for frame_metadata in frames_metadata:
    frame_time.append(frame_metadata['frameTimestamps_sec'])
    # Check if 'auxTrigger0' exists and is not empty
    if trig_chan in frame_metadata and frame_metadata[trig_chan]:
        # Extend the all_auxTrigger0 list with the numbers from this frame's auxTrigger0
        if not auxTrigger:
            pass
        elif frame_metadata['frameNumbers'] in trigger_frame_num or frame_metadata['frameNumbers'] < max(trigger_frame_num)+5:
            continue
        print(frame_metadata)
        if type(frame_metadata[trig_chan]) == list:
            first_float = frame_metadata[trig_chan][0]
        elif type(frame_metadata[trig_chan]) == str:
            match = re.search(r'\d+\.\d+', frame_metadata['auxTrigger1'])
            if match:
                first_float = float(match.group())
            else:
                print(f"Missing trigger for framenum {frame_metadata['frameNumbers']}")
        else:
            print(f"cannot parse trigger for framenum {frame_metadata['frameNumbers']}")
        auxTrigger.append(first_float)
        trigger_frame_num.append(frame_metadata['frameNumbers'])
auxTrigger = np.array(auxTrigger)
len(auxTrigger)

{'frameNumbers': 349, 'acquisitionNumbers': 1, 'frameNumberAcquisition': 349, 'frameTimestamps_sec': 23.08448841, 'acqTriggerTimestamps_sec': -1.0, 'nextFileMarkerTimestamps_sec': -1.0, 'endOfAcquisition': 0, 'endOfAcquisitionMode': 0, 'dcOverVoltage': 0, 'epoch': '[2024 11 27 15 42 33.881]', 'auxTrigger0': [23.1449861], 'auxTrigger1': [], 'auxTrigger2': [], 'auxTrigger3': [], 'I2CData': {}}
{'frameNumbers': 501, 'acquisitionNumbers': 1, 'frameNumberAcquisition': 501, 'frameTimestamps_sec': 33.167491325, 'acqTriggerTimestamps_sec': -1.0, 'nextFileMarkerTimestamps_sec': -1.0, 'endOfAcquisition': 0, 'endOfAcquisitionMode': 0, 'dcOverVoltage': 0, 'epoch': '[2024 11 27 15 42 33.881]', 'auxTrigger0': [33.17034487], 'auxTrigger1': [], 'auxTrigger2': [], 'auxTrigger3': [], 'I2CData': {}}
{'frameNumbers': 652, 'acquisitionNumbers': 1, 'frameNumberAcquisition': 652, 'frameTimestamps_sec': 43.184208615, 'acqTriggerTimestamps_sec': -1.0, 'nextFileMarkerTimestamps_sec': -1.0, 'endOfAcquisition': 0

20

In [ ]:
try: 
    with open(os.path.join(img_path, 'ca_img_wholefile_metadata.json'), 'r') as json_file:
        arm2_img_meta = json.load(json_file)
except:
    raw_img_path = find_tif_file(img_path)
    reader=ScanImageTiffReader(raw_img_path)
    arm2_img_meta = parse_text_to_dict(reader.metadata())
    with open(os.path.join(img_path, 'ca_img_wholefile_metadata.json'), 'w') as json_file:
        json.dump(arm2_img_meta, json_file)